## Finding Roots of a Non-Linear Equation

Consider the equation: $5e^{-x} +x - 5 = 0$ . Suppose we want to solve it with (absolute) accuracy tolerance $\epsilon = 10^{-6}$.

### Exercise 1
Solve using binary search. How many iterations are required?

In [22]:
import numpy as np 


def f(x):
    return 5*np.exp(-x) + x - 5


    #solve for f(x) = 0 ... obviously x=0 is a solution, but lets find the other not zero
    #set initial points
x1 = 4
x2 = 5
target = 10**(-6)
# print(np.sign(f(x1)), np.sign(f(x2)))       #opposite signs 

def midpoint(a, b):
    """
    **Returns:** Midpoint between a and b.
    """
    return 0.5*(a+b)

counter = 1
while np.abs(x1-x2)/(2*counter) > target:
    xn = midpoint(x1, x2)
    if np.sign(f(xn)) == np.sign(f(x1)):
        x1 = xn
    elif np.sign(f(xn)) == np.sign(f(x2)):
        x2 = xn 
    counter = counter+1 #up the iteration counter

print(counter-1, xn, f(xn))
#around 15 iterations are required
    


4 5
15 4.965118408203125 4.030760177187176e-06


### Exercise 2
Solve using relaxation. How many iterations are required?

In [42]:
def f_relax(x):
    return 5 - 5*np.exp(-x)

def fprime(x):
    return -5*np.exp(-x) + 1

x = 1
counter = 1
while np.abs(4.965114231744276-x)/np.abs(1-1/fprime(4.965114231744276)) > target:
    counter = counter + 1
    x = f_relax(x)
    print(x, counter)   
      

print(x, f(x))     
#around 8 iterations


3.1606027941427883 2
4.787999126006694 3
4.958354468938648 4
4.964877613384592 5
4.965105976154342 6
4.96511394374049 7
4.965114221697042 8
4.965114221697042 -9.696728753283423e-09


### Exercise 3
Solve using Newton's Method. How many iterations are required?

In [46]:
x = 5
counter = 1
while np.abs(x - 4.9651142) > target:
    x = x - f(x)/fprime(x)
    counter = counter + 1

print(x, f(x),  counter-1)   
    #around 2-3 iterations, depending on your initial x0
    # ie 3 for x0 = 6, 2 for x0 = 5

4.965114231752603 8.036238341446733e-12 2


### Exercise 4
Solve using Secant Method. How many iterations are required?

In [57]:
x = []
x.append(5)
x.append(5.1)
counter = 1
while np.abs(x[counter] - 4.9651142) > target:
    x1 = x[counter - 1]
    x2 = x[counter]
    x.append(x2 - (f(x2)*(x2  - x1))/(f(x2) - f(x1)))
    counter = counter+1


print(counter, x[counter], f(x[counter]))
    #around 3 iterations


3 4.965114418235045 1.7998489543202822e-07


## Fun with Circuits

### Physics background

![Newman's Exercise 6.5](rescap.png)

Consider the above circuit. Suppose the voltage $V_+$ is time-varying and sinusoidal of the form $V_+ = x_+ \exp(i \omega t)$ with $x_+$ a constant.

The resistors in the circuit can be treated using Ohm's law.
For the capacitors the charge $Q$ and voltage $V$ across them are related by the capacitor law $Q=CV$, where $C$ is the capacitance.
Differentiating both sides of this expression gives the current $I$ flowing in on one side of the capacitor and out on the other:
\begin{equation}
  I = \frac{dQ}{dt} = C \frac{dV}{dt}.
\end{equation}
Now assume the voltages at the points labeled 1, 2, and 3 are of the form $V_1 = x_1 \exp(i \omega t)$, $V_2 = x_2 \exp(i \omega t)$, and $V_3 = x_3 \exp(i \omega t)$.
If you add up the currents using Kirchoff's law that at a junction the sum of the currents in equals the sum of the currents out, you can find that the constants $x_1$,
$x_2$, and $x_3$ satisfy the equations

\begin{align*}
  \left( {1\over R_1} + {1\over R_4} + i\omega C_1 \right) x_1 - i\omega C_1 x_2
  & = {x_+\over R_1}\,, \\
  - i\omega C_1 x_1 + \left( {1\over R_2} + {1\over R_5} + i\omega C_1 + i\omega C_2 \right) x_2 - i\omega C_2 x_3
                                                                                           & = {x_+\over R_2}\,, \\
  - i\omega C_2 x_2 + \left( {1\over R_3} + {1\over R_6} + i\omega C_2 \right) x_3
  & = {x_+\over R_3}\,.
\end{align*}

This is a linear system of equations for three complex numbers, $x_1$, $x_2$, and $x_3$.

We will be solving the above linear system of equations in the form $A x = b$, where $x$ is the vector $(x_1 x_2 x_3$ and $b$ is the vector composed of the right-hand sides of the equations above.

The following function takes as input the list of resistance values ($R_1$ to $R_6$) and the list of capacitances ($C_1$ and $C_2$), and returns (as numpy.array) the matrix $A$. 

In [66]:
def CircuitMatrix(R, C):
    """ I define the matrix A as a function of the one element we turn from a
    resistor to an inductor
    IN: element [complex]: the resistor or inductor
    R [float]: list of resistors R1 to R5
    C [complex]: list of capacitances C1 and C2
    """
    A = np.empty((3, 3), complex)

    # 1st line of matrix
    A[0, 0] = 1./R[0] + 1./R[3] + C[0]
    A[0, 1] = -C[0]
    A[0, 2] = 0.

    # 2nd line of matrix
    A[1, 0] = -C[0]
    A[1, 1] = 1./R[1] + 1./R[4] + C[0] + C[1]
    A[1, 2] = -C[1]

    # 3rd line of matrix
    A[2, 0] = 0.
    A[2, 1] = -C[1]
    A[2, 2] = 1./R[2] + 1./R[5] + C[1]

    return A

And the following function takes as input the list of resistance values and the value of $x+$, and returns (as numpy.array) the vector $b$.

In [80]:
def RHS(R, xplus):
    return xplus*np.array([1./R[0], 1./R[1], 1./R[2]], complex)


### Exercise 5

Use Gaussian Elimination with partial pivoting (see the code fragment below) to solve for $x_1$, $x_2$, and $x_3$. Assume the following:
    
\begin{align*}
  R_1 &= R_3 = R_5 = 1\,\mathrm{k}\Omega, \\
  R_2 &= R_4 = R_6 = 2\,\mathrm{k}\Omega, \\
  C_1 &= 1\,\mu\mathrm{F},\qquad C_2 = 0.5\,\mu\mathrm{F}, \\
  x_+ &= 3\,\mathrm{V},\qquad \omega = 1000~\text{rad/s}.
\end{align*}

Have your program calculate and print, at $t=0$, the amplitudes of the three voltages $|V_1|$, $|V_2|$, and $|V_3|$ and their phases (i.e. the phases of the coefficients $x_1, x_2, x_3$) in degrees.

*Notice that the matrix for this problem has complex elements. You will need to define a complex array to hold it, but your routine should be able to work with real or complex arguments.*

*Hint: the built-in abs() will compute the magnitude, and numpy.angle() will compute the phase of a complex number. You could also use polar and phase from the cmath package.*

In [67]:
import numpy as np
def GaussElim(A_in, v_in, pivot=False):
    """Implement Gaussian Elimination. This should be non-destructive for input
    arrays, so we will copy A and v to temporary variables
    IN:
    A_in [np.array], the matrix to pivot and triangularize
    v_in [np.array], the RHS vector
    pivot [bool, default-False]: user decides if we pivot or not.
    OUT:
    x, the vector solution of A_in x = v_in """
    # copy A and v to temporary variables using copy command
    A = np.copy(A_in)
    v = np.copy(v_in)
    N = len(v)

    for m in range(N):
        if pivot:  # This is where I modify GaussElim
            # compare the mth element to all other mth elements below
            ZeRow = m
            for mm in range(m+1, N):
                if abs(A[mm, m]) > abs(A[ZeRow, m]):
                    ZeRow = mm  # I could swap everytime I find a hit, but that
                    # would be a lot of operations. Instead, I just take note
                    # of which row emerges as the winner

            if ZeRow != m:  # now we know if and what to swap
                A[ZeRow, :], A[m, :] = np.copy(A[m, :]), np.copy(A[ZeRow, :])
                v[ZeRow], v[m] = np.copy(v[m]), np.copy(v[ZeRow])

        # Divide by the diagonal element
        div = A[m, m]
        A[m, :] /= div
        v[m] /= div

        # Now subtract from the lower rows
        for i in range(m+1, N):
            mult = A[i, m]
            A[i, :] -= mult*A[m, :]
            v[i] -= mult*v[m]

    # Backsubstitution
    # create an array of the same type as the input array
    x = np.empty(N, dtype=v.dtype)
    for m in range(N-1, -1, -1):
        x[m] = v[m]
        for i in range(m+1, N):
            x[m] -= A[m, i]*x[i]
    return x

def PartialPivot(A_in, v_in):
    """ see textbook p. 222) """
    return GaussElim(A_in, v_in, True)

In [82]:
R1 = 1
R3 = 1
R5 = 1

R2 = 2
R4 = 2
R6 = 2

C1 = 1
C2 = 0.5

xplus = 3
omega = 1000

V_in = RHS([R1, R2, R3], xplus)

G = CircuitMatrix([R1, R2, R3, R4, R5, R6], [C1, C2])

out = PartialPivot(np.linalg.inv(G), V_in)

print(out)


for i in out:
    print(np.angle(i))

#I wasn't getting complex values, only reals ... middle magnitude was zero hence phase was technically undefined.
#[ 6.00000000e+00+0.j -2.22044605e-16+0.j  5.25000000e+00+0.j]
# 0.0
# 3.141592653589793
# 0.0

[ 6.00000000e+00+0.j -2.22044605e-16+0.j  5.25000000e+00+0.j]
0.0
3.141592653589793
0.0
